In [128]:
import pandas as pd
import numpy as np

# Cargue de datos

In [129]:
# rutas
p1 = "C:/Users/ivan.corredor/2024/IGED_desempeño/"

In [132]:
icfes_f = pd.read_csv(p1 + "RUV_SIMAT_ICFES_FINAL_2024", sep= ";",  low_memory=False) 

In [ ]:
icfes_f["CICLO_VITAL"].value_counts() 
#icfes_f["COLE_DEPTO_UBICACION"].value_counts() 
#icfes_f.info()

In [133]:
icfes_f.columns

Index(['ESTU_TIPODOCUMENTO', 'ESTU_DOCUMENTO', 'ESTU_FECHANACIMIENTO',
       'ESTU_DEPTO_RESIDE', 'ESTU_MCPIO_RESIDE', 'ESTU_GENERO',
       'GRUPO_ETCNICO_SIMAT', 'TIPO_DISCA_SIMAT', 'DISCA_SIMAT', 'EDAD_SIMAT',
       'ESTRATO_SIMAT', 'ESTU_GRADO', 'COLE_NOMBRE_ESTABLECIMIENTO',
       'COLE_NOMBRE_SEDE', 'COLE_CALENDARIO', 'COLE_GENERO', 'COLE_JORNADA',
       'COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'COLE_COD_DEPTO_UBICACION',
       'COLE_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_MCPIO_UBICACION', 'HV5', 'VICTIMA', 'NO_VICTIMA', 'PUNT_GLOBAL',
       'PERCENTIL_GLOBAL'],
      dtype='object')

In [136]:
## observar si existen datos para no víctimas y víctimas por variable
icfes_f.groupby("VICTIMA")["DISCA_SIMAT"].apply(lambda x: np.sum(~x.isnull()))

VICTIMA
0    463314
1    104592
Name: DISCA_SIMAT, dtype: int64

In [137]:
icfes_f["COLE_MCPIO_UBICA2"]= icfes_f["COLE_MCPIO_UBICACION"].replace("[' '-.]", '', regex =True)

# General

<img src="formula_G_Hedges.png" width="500" height="400">

## Media, tamaño universo y varianza

In [138]:
# Estadísticas necesarias
grl_1 = icfes_f.groupby(["VICTIMA"]).apply(
        lambda x: pd.Series({
        'MEDIA': round(x['PUNT_GLOBAL'].mean(), 1),
        'UNIVERSO': round(x['PUNT_GLOBAL'].size, 1),
        'VARIANZA': round(x['PUNT_GLOBAL'].var(), 1)
    })
    ).reset_index()
grl_1 

,VICTIMA,MEDIA,UNIVERSO,VARIANZA
0,0,257.5,478879.0,2760.4
1,1,234.3,104592.0,2111.4


## Numerador, denominador, G de hedges

In [139]:
# crear data frame para el númerador y denominador
grl_2 = pd.DataFrame({"NUMERADOR": [grl_1.loc[0, "MEDIA"] - grl_1.loc[1, "MEDIA"]],
                      "DENOMINADOR": 
                      [
                      np.sqrt(
                                (
                                    (
                                        (grl_1.loc[0, "UNIVERSO"] - 1)* grl_1.loc[0, "VARIANZA"]
                                    )
                                    + (
                                        (grl_1.loc[1, "UNIVERSO"] - 1) * grl_1.loc[1, "VARIANZA"]
                                    )
                                ) 
                                / (
                                    grl_1.loc[0, "UNIVERSO"] +
                                   grl_1.loc[1, "UNIVERSO"] 
                                   - 2
                                )
                                )
                      ]
                     })
grl_2["INDICADOR"]  = grl_2["NUMERADOR"] / grl_2["DENOMINADOR"]

grl_2["INDICADOR_AJUSTADO"]  = grl_2["INDICADOR"] * ( 1 - 
                                                        (3 / (
                                                                (
                                                                    4 * (grl_1.loc[0, "UNIVERSO"] + grl_1.loc[1, "UNIVERSO"])
                                                                ) - 1
                                                        )
                                                    )
                                                    )
grl_2

#grl = grl.stack()
#grl = grl.unstack(0)

,NUMERADOR,DENOMINADOR,INDICADOR,INDICADOR_AJUSTADO
0,23.2,51.420444,0.451182,0.451182


# Generalización por enfoque

## Estadísticos

In [147]:
# Variables
names_var = ["ESTU_GENERO", "GRUPO_ETCNICO_SIMAT", "DISCA_SIMAT",
             "ESTRATO_SIMAT", 
             "COLE_NATURALEZA", "COLE_AREA_UBICACION", 
             "COLE_DEPTO_UBICACION", "COLE_MCPIO_UBICA2"]#"EDAD_SIMAT", #"ESTU_GRADO",

# tablas de salida
tab_out = ["sex", "etni", "disca", 
            "estrato", 
           "nat", "zona", "dpto", "mun"] #"edad", #"grado",

# lista para almacenar las listas de variables
ll_var = []

# Estadísticas necesarias
for g, h in zip(names_var, tab_out):

    df = icfes_f.groupby(["VICTIMA", g]).apply(
        lambda x: pd.Series({
            "MEDIA": round(x["PUNT_GLOBAL"].mean(), 1),
            "UNIVERSO": round(x["PUNT_GLOBAL"].size, 1),
            "VARIANZA": round(x["PUNT_GLOBAL"].var(), 1)
        })
    )
    # Excepción: subselect de muni con no víctimas y víctimas
    # obetener valores únicos del nivel 1 del multiindex
    if g == "COLE_MCPIO_UBICA2":
        #filtrar df para municipios con > 3 víctimas
        df = df[df["UNIVERSO"] > 3]
        paso_1 = df.index.get_level_values('VICTIMA').unique()
        # to calculate the number of unique values from the first level for each group in the second level.
        paso_2 = df.groupby(level='COLE_MCPIO_UBICA2').apply(lambda x: x.index.get_level_values(0).nunique())
        # obtiene indices si el valor de paso_2 (cuenta de municipios) = 2 (longitud nivel 1)
        paso_3 = paso_2[paso_2 == len(paso_1)].index.tolist()
        # filtra df donde valores del nivel 2 (multiindex) estén en indices de paso_3
        df = df[df.index.get_level_values('COLE_MCPIO_UBICA2').isin(paso_3)]
    else:
        pass
    # se crean dfs tab_out_1: 
    exec(f"{h}_1 = df.copy()")
    # se crean listas de variables l_tab_out_1:
    exec(f"l_{h}_1 = list(filter(pd.notna, list(df.index.get_level_values(g).unique())))")
    # lista para almacenar las listas anteriores
    # eval is used to evaluate a single Python expression or a small block of code represented as a string. 
    l_var = eval(f"l_{h}_1")
    ll_var.append(l_var)

## Estimadores - G Hedges y  ajustado

In [148]:
# lista con nombres de variables de salida
var_out = ["SEXO", "ETNIA", "DISCAPACIDAD", 
            "ESTRATO",  
           "NATURALEZA", "ZONA", "DEPARTAMENTO", "MUNICIPIO"] #"EDAD",#"GRADO",

# bucle que recorre índice y dataframes
for k_indx, k in enumerate(tab_out):
    # Retrieve the DataFrame using eval() 
    df2 = eval(f"{k}_1")
    current_vars = ll_var[k_indx]   
    # lista con tantos nombres de variables de salida como categorias tenga la variable la tabla madre
    var_df = [
        # Repeat 'value' len(sublist) times for each element in sublist
        [value] * len(sublist) for value, sublist in zip(var_out, ll_var)
             ]
    # indexar nombres de variable a su datafrane respectivo
    names_current_vars = var_df[k_indx]  
    # Se crean listas vacias l_dtab_out
    exec(f"l_d{k} = []")
    
    # recorre dos listas (categorias var) y (nombres var)
    for i, m in zip(current_vars, names_current_vars):
        # indices para dataframes tab_out_1
        colnv = (0, i)  
        colvic = (1, i)
        # crea formula
        numerador = df2.loc[colnv, "MEDIA"] - df2.loc[colvic, "MEDIA"]
        denominador = np.sqrt(
                    (
                        (
                            (df2.loc[colnv, "UNIVERSO"] - 1)* df2.loc[colnv, "VARIANZA"]
                        )
                        + (
                            (df2.loc[colvic, "UNIVERSO"] - 1) * df2.loc[colvic, "VARIANZA"]
                        )
                    ) 
                    / (
                        df2.loc[colnv, "UNIVERSO"] +
                       df2.loc[colvic, "UNIVERSO"] 
                       - 2
                    )
                        )
        # crea dataframe con variable asociada a cada dataframes de tab_out_1
        ds = pd.DataFrame({
            f"{m}": [i],
            "NUMERADOR": [numerador],
            "DENOMINADOR": [denominador]
        })
        # Append dfs to the list
        eval(f"l_d{k}").append(ds)
    # se crean dfs tab_out_2: 
    exec(f'''{k}_2 = pd.concat(eval("l_d{k}"), axis=0, ignore_index=True)''')
    # calcula G de Hedges
    exec(f'''{k}_2["INDICADOR"]  = {k}_2["NUMERADOR"] / {k}_2["DENOMINADOR"]''')
    # listas vacias l_ind_adjtab_out para indicador ajustado
    exec(f"l_ind_adj{k} = []")
    # recupera el valor del objeto string, en este caso un dataframe
    df3 = eval(f"{k}_2")
    for i, m in zip(current_vars, names_current_vars):
        # indices para dfs tab_out_2
        colnv = (0, i)  
        colvic = (1, i)
        # calcula indicador ajustado
        ind_ajustado = df3.set_index(m).loc[i, "INDICADOR"] * ( 1 - 
                                        (3 / (
                                                (
                                                4 * (df2.loc[colnv, "UNIVERSO"] + 
                                                     df2.loc[colvic, "UNIVERSO"])
                                                ) - 1
                                            )
                                        )
                                     )
        # crea df con variable y categoria asociadas a dfs de tab_out_1 y tab_out_2
        ds2 = pd.DataFrame({
            f"{m}": [i],
            "INDICADOR_AJUS": [ind_ajustado]
        })
        # crea lista con dfs l_ind_adjtab_out
        eval(f"l_ind_adj{k}").append(ds2)
    # se crean dfs tab_out_2b: 
    exec(f'''{k}_2b = pd.concat(eval("l_ind_adj{k}"), axis=0, ignore_index=True)''')
    # se crean dfs tab_out_3, concatenando tab_out_2 y tab_out_2b. Se elimina índice
    exec(f'''{k}_3 = pd.concat([eval(f"{k}_2"), eval(f"{k}_2b")], axis=1).reset_index(drop=True)''')
    # se crean otros dfs para evitar interrupciones cuando se corra el código en bloque más de una vez
    exec(f'''{k}_4 = eval(f"{k}_3").loc[:, ~eval(f"{k}_3").columns.duplicated()]''')

## Etiquetar variables

In [149]:
#sexo
sex_4.loc[:,"SEXO"] = sex_4["SEXO"].replace(["F", "M"], 
                    ["Mujer", "Hombre"])
#etnia
etni_4.loc[:,"ETNIA"] = etni_4["ETNIA"].replace([0,1,2,3,4,5,6,7], 
                                    ["No aplica", "Indígenas", 
                                    "Negritudes", "Rrom", 
                                     "Otras Étnias", 
                                     "Raizales", "Afrodescendientes",
                                     "Palenqueros"])

## Agregar codigo divipola

In [42]:
#pd.set_option('display.max_columns', None)

In [150]:
## dejar solo la tabla de valor únicos de ifces por municipio
reduc_icfes = icfes_f[["COLE_COD_DEPTO_UBICACION","COLE_DEPTO_UBICACION",
                      "COLE_COD_MCPIO_UBICACION", "COLE_MCPIO_UBICA2"]]
# mun
dane_mun_icf = reduc_icfes.drop_duplicates(subset=['COLE_MCPIO_UBICA2'])
# dpto
dane_dpto_icf = reduc_icfes.drop_duplicates(subset=['COLE_COD_DEPTO_UBICACION'])
# merge mun
mun_4 = mun_4.merge(dane_mun_icf[["COLE_COD_MCPIO_UBICACION", "COLE_MCPIO_UBICA2"]],
            how= "left", left_on= "MUNICIPIO", right_on = "COLE_MCPIO_UBICA2",
            validate= "1:m")
# drop column
mun_4.drop(columns="COLE_MCPIO_UBICA2", inplace=True)
# add zero to left
mun_4["COLE_COD_MCPIO_UBICACION"] = pd.to_numeric(mun_4["COLE_COD_MCPIO_UBICACION"], downcast="integer").astype(str).str.zfill(5)
# reorder
mun_4 = mun_4[['COLE_COD_MCPIO_UBICACION','MUNICIPIO', 'NUMERADOR', 
    'DENOMINADOR', 'INDICADOR', 'INDICADOR_AJUS']]
# merge dpto
dpto_4 = dpto_4.merge(dane_dpto_icf[["COLE_COD_DEPTO_UBICACION","COLE_DEPTO_UBICACION"]],
            how= "left", left_on= "DEPARTAMENTO", right_on = "COLE_DEPTO_UBICACION",
            validate= "1:m")
# drop column 
dpto_4.drop(columns="COLE_DEPTO_UBICACION", inplace=True)

# eliminate last zero to rigth and add zero to left
def remove_last_zero(s):
    s = str(s)  # Ensure the value is a string
    if s.endswith('0'):
        return s[:-1]  # Remove the last character
    return s

dpto_4['COLE_COD_DEPTO_UBICACION'] = dpto_4['COLE_COD_DEPTO_UBICACION'].apply(remove_last_zero).astype(str).str.zfill(2)
# reorder
dpto_4 = dpto_4[['COLE_COD_DEPTO_UBICACION', 'DEPARTAMENTO', 'NUMERADOR',
             'DENOMINADOR', 'INDICADOR','INDICADOR_AJUS']]

## Export

In [151]:
# ruta de salida
p2 = "C:/Users/ivan.corredor/2024/IGED_desempeño/salidas"

In [152]:
salida = p2 + "/res_desempeño_2024.xlsx"
writer = pd.ExcelWriter(salida)

# Generar las tablas y nombres de pestañas de acuerdo con el número de enfoques.
# iniciando por la tabla general
tablas = [grl_2]
pestañas = ["General"]

for k in tab_out:
    # nombre de tablas y pestañas
   #The extend() method allows you to append elements from an iterable (like a list)
   #to the end of another list.
    tablas.extend([eval(f"{k}_1"), eval(f"{k}_4")])
    pestañas.extend([f"{k}_1", f"{k}_4"])

#exportar cada tabla como una pestaña del archivo excel
for i,j in zip(tablas, pestañas):
    i.to_excel(writer, sheet_name = j, header = True, index= True)

writer.close()